In [44]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.layers import Bidirectional
from utils import load_short_jokes_corpus
import numpy as np
import random
import sys

In [56]:
jokes, text = load_short_jokes_corpus(limit=20000)

chars = sorted(list(set(text)))
words = set(text.split(' '))

In [57]:
print 'Unique words: {w}'.format(w=len(words))

Unique words: 42480


In [58]:
word_indices = dict((c, i) for i, c in enumerate(words))
indices_word = dict((i, c) for i, c in enumerate(words))

In [59]:
maxlen = 5
step = 1
sentences = []
next_words = []
next_words = []
sentences1 = []
list_words = []

sentences2 = []
list_words = text.lower().split()

In [60]:
for i in range(0,len(list_words)-maxlen, step):
    sentences2 = ' '.join(list_words[i: i + maxlen])
    sentences.append(sentences2)
    next_words.append((list_words[i + maxlen]))
print 'Vectorization...'
X = np.zeros((len(sentences), maxlen, len(words)), dtype=np.bool)
y = np.zeros((len(sentences), len(words)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence.split()):
        #print(i,t,word)
        X[i, t, word_indices[word]] = 1
    y[i, word_indices[next_words[i]]] = 1

Vectorization...


In [61]:
model = Sequential()
model.add(Bidirectional(LSTM(512, return_sequences=True), input_shape=(maxlen, len(words))))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(512, return_sequences=False)))
model.add(Dropout(0.2))
model.add(Dense(len(words)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [51]:
print model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_8 (Bidirection (None, 5, 1024)           5603328   
_________________________________________________________________
dropout_7 (Dropout)          (None, 5, 1024)           0         
_________________________________________________________________
bidirectional_9 (Bidirection (None, 1024)              6295552   
_________________________________________________________________
dropout_8 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 855)               876375    
_________________________________________________________________
activation_4 (Activation)    (None, 855)               0         
Total params: 12,775,255
Trainable params: 12,775,255
Non-trainable params: 0
________________________________________________________________

In [54]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds.astype('float64'))
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
for iteration in range(1, 25):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=2)
    model.save_weights('jokesWeights',overwrite=True)